# 🧪 ORM Sandbox

This notebook is used for **testing SQLAlchemy ORM classes** during development.  
Used for:

- Create and query model instances  
- Insert test data into the database  
- Experiment with filters and relationships  
- Verify behaviours before integrating into the main app


## 🏷️ Product Class

In [ ]:
from app import create_app
from extensions import db
from models.product import Product, Batch
from models.store_model import Order, OrderItem
from models.main_model import ContactMessage
from datetime import datetime

app = create_app()
app.app_context().push()

In [ ]:
# For adding a product to the database
from datetime import datetime

new_product = Product(
    sku="Test",
    name="Test",
    description="Sliced cucumbers in sugar and vinegar",
    price=6.99,
    size="576 g",
    category="Pickles",
    ingredients="Cucumber, dill, chilli, bayleaf, sugar, vinegar",
    featured=True,
    image_main="images/products/pickle006-01.jpeg",
    image_1="images/products/pickle006-02.jpeg",
    image_2="",
    active=True,
    packs_per_box=6
)

db.session.add(new_product)
db.session.commit()


In [ ]:
# For querying specific SKU
product = Product.query.filter_by(sku="SKU001").first()
print(f"Product: {product.name}, Stock: {product.stock_quantity}")


In [ ]:
# For querying all SKU's
products = Product.query.all()

for product in products:
    print(f"SKU: {product.sku}, Name: {product.name}, Stock: {product.stock_quantity}, Price: {product.price}")


In [ ]:
# Search products by word in name
from sqlalchemy import or_

search_term = "spicy"

matching_products = Product.query.filter(Product.name.ilike(f"%{search_term}%")).all()

for product in matching_products:
    print(f"SKU: {product.sku}, Name: {product.name}, Price: {product.price}")


In [ ]:
#Filtering products by a maximum price
max_price = 7.00

matching_products = Product.query.filter(Product.price < max_price).all()

for product in matching_products:
    print(f"SKU: {product.sku}, Name: {product.name}, Price: {product.price}")

In [ ]:
#Filtering products by a minimum price
min_price = 7.00

matching_products = Product.query.filter(min_price < Product.price).all()

for product in matching_products:
    print(f"SKU: {product.sku}, Name: {product.name}, Price: {product.price}")

In [ ]:
# Test Product delete
product = Product.query.filter_by(sku="SKU001").first()

if product:
    db.session.delete(product)
    db.session.commit()
    print(f"Deleted product with SKU: {product.sku}")
else:
    print("Product not found.")

In [ ]:
#Test Add Batch
from datetime import datetime

new_batch = Batch(
    stock_quantity ="123",
    stock_location ="Sandbox Test",
    expiry_date =datetime(2026, 8, 3),
    product_sku ="pickle005"
)

db.session.add(new_batch)
db.session.commit()

In [ ]:
#Test Batch Delete
batch = Batch.query.filter_by(id="12").first()

if batch:
    db.session.delete(batch)
    db.session.commit()
    print(f"Deleted batch with ID: {batch.id}")
else:
    print("Product not found.")

SQL for changing the name of the in_stock column to active

In [ ]:
ALTER TABLE products RENAME COLUMN in_stock TO active;

In [ ]:
#Test Add Order

test_order = Order(
    first_name="Joe",
    surname="Smith",
    email="alice@example.com",
    phone_number="0871234567",
    address_line1="123 Main Street",
    address_line2="Apt 4B",
    city="Dublin",
    county="Dublin",
    postcode="D01 AB23",
    country="Ireland"
)

db.session.add(test_order)
db.session.commit()

print(f"Order added: {test_order}")


In [ ]:
#For querying all Orders
orders = Order.query.all()

for order in orders:
    print(f"Order Ref: {order.order_reference}")
    print(f"Customer: {order.first_name} {order.surname}")
    print(f"Email: {order.email}, Phone: {order.phone_number}")
    print(f"Address: {order.address_line1}, {order.address_line2}, {order.city}, {order.county}, {order.postcode}, {order.country}")
    print(f"Created At: {order.created_at}")
    print(f"Status: {order.status}")
    print("-----------------------------------")


In [ ]:
#Test Add single Order Item

order_id = 2
product_sku = "pickle002"
quantity = 30


order = Order.query.get(order_id)
product = Product.query.get(product_sku)

if order and product:
    order_item = OrderItem(
        order_id=order.id,
        product_sku=product.sku,
        quantity=quantity
    )

    
    db.session.add(order_item)
    db.session.commit()

    print(f"Added {quantity} x {product.name} to Order {order.order_reference}")
else:
    print("Invalid order ID or product SKU.")


In [ ]:
#Test Add multiple Order Items

order_id = 3

items_to_add = [
    {"product_sku": "pickle002", "quantity": 32},
    {"product_sku": "pickle004", "quantity": 12},
    {"product_sku": "pickle005", "quantity": 10},
]

order = Order.query.get(order_id)


if order:
    for item in items_to_add:
        product = Product.query.get(item["product_sku"])
        if product:
            order_item = OrderItem(
                order_id=order.id,
                product_sku=product.sku,
                quantity=item["quantity"]
            )
            db.session.add(order_item)
            print(f"Added {item['quantity']} x {product.name} to Order {order.order_reference}")
        else:
            print(f" SKU {item['product_sku']} not found.")

    db.session.commit()
else:
    print(f" Order ID {order_id} not found.")


In [ ]:
#For Querying all OrderItems
order_items = OrderItem.query.all()

for item in order_items:
    order = item.order
    product = item.product
    print(f"Order Ref: {order.order_reference} | Customer: {order.first_name} {order.surname}")
    print(f"SKU: {item.product_sku} | Product Name: {product.name} | Quantity: {item.quantity}")
    print("-----------------------------------")

In [ ]:
#For adding Contact Messages
new_message = ContactMessage(
    name="Marek Novak",
    email="marek.n@example.com",
    subject="Great Service",
    message="Just wanted to say thank you for the fast delivery and excellent packaging.",
    submitted_at=datetime.utcnow()
)

db.session.add(new_message)
db.session.commit()

In [ ]:
#For Querying all Contact Messages
all_messages = ContactMessage.query.all()
for msg in all_messages:
    print(f"{msg.id}: {msg.subject} from {msg.name} on {msg.submitted_at}")
